In [1]:
!pip install flask-ngrok

In [2]:
pip install tokenizers

     |████████████████████████████████| 3.3 MB 25.1 MB/s 


In [14]:
from flask import Flask, jsonify, request, render_template
from flask_ngrok import run_with_ngrok
import pandas as pd
import json
import requests

In [15]:
import random

In [16]:
from flask import Flask, send_file
from flask_ngrok import run_with_ngrok

In [17]:
from pathlib import Path

from tokenizers import normalizers
from tokenizers.normalizers import NFKC, Lowercase
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import WordPieceTrainer

In [18]:
paths = [str(x) for x in Path("/content/drive/MyDrive/Cursos_UNI/Analitica_de_datos/Final/corpus").glob("**/*.txt")]

In [19]:
bert_tokenizer = Tokenizer(WordPiece(unk_token= "[UNK]"))
bert_tokenizer.normalizer = normalizers.Sequence([NFKC(), Lowercase()])
bert_tokenizer.pre_tokenizer = Whitespace()

bert_tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", 1),
        ("[SEP]", 2),
    ],
)


# Training model

trainer = WordPieceTrainer(
    vocab_size=50265, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

bert_tokenizer.train(paths, trainer)
bert_tokenizer.save("quechuabert")

In [27]:
# Módulo de reconocimiento de entidad País
#Paises = {'Mishiku': 'Mishiku llaqta', 'Fransiapa': 'París', 'Ispaña': 'Madrid', 'Españapa': 'Madrid', 'Inglaterra': 'Londres', 'Italya': 'Roma',}
Paises = {'México': 'México', 'Francia': 'París', 'España': 'Madrid', 'Inglaterra': 'Londres', 'Italia': 'Roma'}
Resp_Paises = ['wan mayna allinmi sumaq kay killakunapim watakama', #y es hermoso en esta época del año 
               'Munaymanm yachay rikuy', #me encantaría poder visitarla
               'puriy Chatbots yachay watuku nacionkunapiqa yanqa qawanapaq'] #lástima que los ChatBots solo podemos hacer visitas a paises de manera virtual
Paises_Unknown = ['puriy mana  yachayanichu uma llaqta nacionkunapiqa, Rishaq estudiay #Chatbot saqra', #Lo siento pero creo que no conozco la capital de ese país, creo que debo estudiar más #Mal ChatBot
                  'Pampachakuy, ama Chusay Llahta, ama Ruwaysiy mayqantaq Llahta', # Creo que dirás que soy chafa, pero no me sé esa capital, pero pregunta por otra   
                  'Puriy mana  yachayanichu estudiay allpa saywachi yanapayuq chaykunaqa'] #Creo que me pondré a estudiar Geografía, para que pueda responderte en un futuro 
Inicio = ['Sumaq chisi ', #Buenas Tardes
          'allin suka yachachak ', #Hola como estas
          'Miski simiwan ', #La respuesta
          'allin puchay ', #Buenos dias
          'Uma llanqta ']#Capital
def Country(inp):
    pais_act = 0
    for pais in Paises.keys():
        if inp.lower().count(pais.lower()) > 0:
            pais_act = 1
            print('\nChatBot: ' + 'Uma llanqta ' + str(pais) + ' kay ' + str(Paises.get(pais)) + ' ' + random.choice(Resp_Paises) + '[Entidad]\n')
    if pais_act == 0:
        print('\nChatBot: ' + random.choice(Paises_Unknown) + '\n')

In [ ]:
col_names = ['Pais','Capital']
data=pd.read_csv("/content/drive/MyDrive/Cursos_UNI/Analitica_de_datos/Final/Paises_quechua.csv", header=None, names=col_names, delimiter=';')
data.head()

In [22]:
lista_capital=[]
lista_pais=[]
for i in data.Capital:
  lista_capital.append(i)

for j in data.Pais:
  lista_pais.append(j)

lista_pais

['bolivia', 'francia', 'mishiku', 'italia', 'españa', 'peru']

In [ ]:
app = Flask(__name__, template_folder="/content/drive/MyDrive/Cursos_UNI/Analitica_de_datos/Final")
run_with_ngrok(app)


@app.route("/")
def hello_world():
  return render_template("home.html")

@app.route("/get") 
def get_bot_response():
  usertext = request.args.get('msg')
  entrada = bert_tokenizer.encode(usertext)
  pais_act=0
  for i in entrada.tokens:
    if i in lista_pais:
      pais_act=1
      inde=lista_pais.index(i)
      return str(random.choice(Inicio) + i + ' kay ' + lista_capital[inde] + '  ' + random.choice(Resp_Paises) + '\n')
  if pais_act==0:
    return str(random.choice(Paises_Unknown) +  '\n')




  ##return usertext

app.run()